In [16]:
import duckdb
from market_streaming.domain.models import MarketTick

In [17]:
from pathlib import Path
PROJECT_ROOT = Path.cwd().parents[0]  # adjust if you launch Jupyter from root
DB_PATH = PROJECT_ROOT / "data" / "market_data.duckdb"

con = duckdb.connect(str(DB_PATH))
con.execute("SELECT * FROM market_ticks LIMIT 10").fetchdf()

,ts,symbol,price,volume
0,2025-12-01 09:00:00,SP500,4710.04,323264
1,2025-12-01 09:05:00,SP500,4718.01,375005
2,2025-12-01 09:10:00,SP500,4649.09,381444
3,2025-12-01 09:15:00,SP500,4641.73,341846
4,2025-12-01 09:20:00,SP500,4704.77,202593
5,2025-12-01 09:25:00,SP500,4722.25,381851
6,2025-12-01 09:30:00,SP500,4651.30,264484
7,2025-12-01 09:35:00,SP500,4704.39,268830
8,2025-12-01 09:40:00,SP500,4734.95,347637
9,2025-12-01 09:45:00,SP500,4650.66,457487


In [18]:
con.execute("""
    SELECT symbol, COUNT(*) AS n_rows
    FROM market_ticks
    GROUP BY symbol
    ORDER BY n_rows DESC
""").fetchdf()

,symbol,n_rows
0,NIKKEI225,50
1,SP500,50
2,STOXX600,50


In [14]:
con.close()